In [1]:
import os
import osmnx as ox
import rasterio
from shapely.geometry import box
import geopandas as gpd
import rasterio.features
from shapely.ops import unary_union
from shapely.geometry import shape

# 1. Defining Parameters

In [ ]:
os.environ['GDAL_FILENAME_IS_UTF8'] = 'NO'
os.environ['SHAPE_ENCODING'] = "utf-8"
os.environ['CPL_ZIP_ENCODING'] = "UTF-8"

path_dtm = r".\Cut_data\dtm.tif"
output_graphml = r".\Raw_graph\raw_street_graph.graphml"
output_gpkg = r".\Raw_graph\raw_street_graph.gpkg"

# 2. Reading DTM and Vetorizing with Mask

- Read the Digital Terrain Model
- Loads the Model Mask
- Drop pixels outside the mask (No Data points)

In [ ]:
with rasterio.open(path_dtm) as src:
    # Origin CRS
    crs_origem = src.crs

    mask_shapes = rasterio.features.shapes(
        src.read_masks(1),
        transform=src.transform,
        mask=src.read_masks(1) 
    )
    
    geometries = [shape(geom) for geom, val in mask_shapes]
    
    polygonorig_crs = unary_union(geometries)

# Converting final geometry to Lat/Lon (EPSG:4326)
reference_area = gpd.GeoSeries([polygonorig_crs], crs=crs_origem).to_crs(epsg=4326).iloc[0]

# 3. Downloading the streets graph using the DTM geometry

In [ ]:
G = ox.graph_from_polygon(reference_area, network_type='all')

print(f"Graph with: {len(G.nodes)} nodes and {len(G.edges)} edges.")

# 4. Saving the data

In [ ]:
ox.save_graphml(G, filepath=output_graphml)
ox.save_graph_geopackage(G, filepath=output_gpkg)